# Filtragem dos dados do bango de dados

In [5]:
import pandas as pd
import numpy as np
import os
import datetime

# Froms
from gcpUtils.auth import getCredentials
from gcpUtils.bigQuery import pandasToBq, tableToPandas
from gcpUtils.google_storage_manager import *

cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")

In [ ]:
# caso 1

query = """
WITH base AS (
    SELECT
        FILIAL,
        DATA,
        -- Se existir algum produto com ressuprimento ≠ 0, a filial NÃO está em 0 geral.
        COUNTIF(RESSUPRIR <> 0) AS produtos_com_ressuprimento
    FROM `planejamento-animale-292719.checklists_rollout.ANIMALE_checklist`
    WHERE DATA > '2025-01-15' 
      AND DATA < '2025-10-06'
    GROUP BY FILIAL, DATA
),

-- Marca os dias em que a filial está completamente sem ressuprimento
status_filial AS (
    SELECT
        FILIAL,
        DATA,
        CASE WHEN produtos_com_ressuprimento = 0 THEN 1 ELSE 0 END AS sem_ressuprimento
    FROM base
),

-- Detecta o início de cada período sem ressuprimento
marcacao_inicio AS (
    SELECT
        FILIAL,
        DATA,
        sem_ressuprimento,
        CASE
            WHEN sem_ressuprimento = 1
            # LAG olha a linha anterior (dia anterior da mesma filial), se for 0, é o início de um novo período
            # OVER define a partição (filial) e a ordem (data)
             AND LAG(sem_ressuprimento, 1, 0) OVER (PARTITION BY FILIAL ORDER BY DATA) = 0
            THEN 1 ELSE 0
        END AS inicio_periodo
    FROM status_filial
),


-- Cria um ID de grupo (período) cumulativo
grupos AS (
    SELECT
        FILIAL,
        DATA,
        sem_ressuprimento,
        SUM(inicio_periodo) OVER (
            PARTITION BY FILIAL ORDER BY DATA ROWS UNBOUNDED PRECEDING
        ) AS periodo_id
    FROM marcacao_inicio
    WHERE sem_ressuprimento = 1
),

-- Resume cada período com início e fim
resumo AS (
    SELECT
        FILIAL,
        periodo_id AS ID_PERIODO,
        MIN(CAST(DATA AS DATE)) AS inicio_sem_ressuprimento,
        MAX(CAST(DATA AS DATE)) AS fim_sem_ressuprimento,
        DATE_DIFF(MAX(CAST(DATA AS DATE)), MIN(CAST(DATA AS DATE)), DAY) + 1 AS dias_sem_ressuprimento
    FROM grupos
    GROUP BY FILIAL, periodo_id
)

SELECT
    r.*,
    -- caso o período ainda esteja ativo (última data da base = fim)
    CASE 
        WHEN r.fim_sem_ressuprimento = (
            SELECT MAX(CAST(DATA AS DATE)) FROM status_filial s WHERE s.FILIAL = r.FILIAL
        )
        THEN 'EM ANDAMENTO'
        ELSE 'FINALIZADO'
    END AS STATUS
FROM resumo r
ORDER BY FILIAL, ID_PERIODO;

"""


df_prod_desc = tableToPandas(query, 'planejamento-animale-292719', cred)
df_prod_desc.columns = df_prod_desc.columns.str.upper()

In [19]:
# caso 2


query = """
WITH base AS (
    SELECT
        FILIAL,
        DATA,
        COUNTIF(RESSUPRIR <> 0) AS produtos_com_ressuprimento
    FROM `planejamento-animale-292719.checklists_rollout.ANIMALE_checklist`
    WHERE DATA > '2025-01-15' 
      AND DATA < '2025-10-06'
      AND FILIAL LIKE '%CM%'
    GROUP BY FILIAL, DATA
),

status_filial AS (
    SELECT
        FILIAL,
        DATA,
        CASE WHEN produtos_com_ressuprimento = 0 THEN 1 ELSE 0 END AS sem_ressuprimento
    FROM base
),

-- Detecta o início de cada período sem ressuprimento
marcacao_inicio AS (
    SELECT
        FILIAL,
        DATA,
        sem_ressuprimento,
        CAST(
            CASE
                WHEN sem_ressuprimento = 1
                 AND LAG(sem_ressuprimento, 1, 0) OVER (PARTITION BY FILIAL ORDER BY DATA) = 0
                THEN 1 ELSE 0
            END AS INT64
        ) AS inicio_periodo
    FROM status_filial
),

-- Cria IDs de grupos contínuos
grupos AS (
    SELECT
        FILIAL,
        DATA,
        sem_ressuprimento,
        SUM(inicio_periodo) OVER (
            PARTITION BY FILIAL ORDER BY DATA ROWS UNBOUNDED PRECEDING
        ) AS periodo_id
    FROM marcacao_inicio
    WHERE sem_ressuprimento = 1
),

-- Resume períodos
resumo AS (
    SELECT
        FILIAL,
        periodo_id AS ID_PERIODO,
        MIN(CAST(DATA AS DATE)) AS inicio_sem_ressuprimento,
        MAX(CAST(DATA AS DATE)) AS fim_sem_ressuprimento,
        DATE_DIFF(MAX(CAST(DATA AS DATE)), MIN(CAST(DATA AS DATE)), DAY) + 1 AS dias_sem_ressuprimento
    FROM grupos
    GROUP BY FILIAL, periodo_id
),

-- Classifica cada período e define se ainda está ativo
classificado AS (
    SELECT
        r.*,
        CASE 
            WHEN r.fim_sem_ressuprimento = (
                SELECT MAX(CAST(DATA AS DATE)) 
                FROM status_filial s WHERE s.FILIAL = r.FILIAL
            )
            THEN 'EM ANDAMENTO'
            ELSE 'FINALIZADO'
        END AS STATUS,
        CASE 
            WHEN r.dias_sem_ressuprimento > 30 THEN 'SUPERIOR A 30 DIAS'
            ELSE 'ATE 30 DIAS'
        END AS CLASSIFICACAO
    FROM resumo r
),

-- Agrupa filiais que tiveram ou não algum período > 30 dias
agrupado AS (
    SELECT
        FILIAL,
        MAX(CASE WHEN CLASSIFICACAO = 'SUPERIOR A 30 DIAS' THEN 1 ELSE 0 END) AS teve_periodo_maior_30
    FROM classificado
    GROUP BY FILIAL
)

-- Resultado final:
SELECT 
    a.FILIAL,
    CASE 
        WHEN a.teve_periodo_maior_30 = 1 THEN 'FILIAL COM PERÍODO > 30 DIAS'
        ELSE 'FILIAL SEM PERÍODO > 30 DIAS'
    END AS STATUS_FILIAL,
    c.ID_PERIODO,
    c.inicio_sem_ressuprimento,
    c.fim_sem_ressuprimento,
    c.dias_sem_ressuprimento,
    c.CLASSIFICACAO,
    c.STATUS
FROM agrupado a
LEFT JOIN classificado c
    ON a.FILIAL = c.FILIAL
ORDER BY a.FILIAL, c.ID_PERIODO;

"""

df_prod_desc = tableToPandas(query, 'planejamento-animale-292719', cred)
df_prod_desc.columns = df_prod_desc.columns.str.upper()

In [20]:
# caso 1

local_path = os.path.join(os.getcwd(), 'data')
if not os.path.exists(local_path):
    os.makedirs(local_path)
file_path = os.path.join(local_path, 'ressuprimento_filiais.xlsx')
df_prod_desc.to_excel(file_path, index=False)